In [6]:
# import pickle

# X_pathname = "/Volumes/MasterDrive/ML project data/X.pkl"
# with open(X_pathname, 'rb') as infile_X:
#     X = pickle.load(infile_X)
    
# y_pathname = "/Volumes/MasterDrive/ML project data/y.pkl"
# with open(y_pathname, 'rb') as infile_y:
#     y = pickle.load(infile_y)

In [7]:
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.decomposition import PCA

# Xtr, Xts, ytr, yts = train_test_split(X, y, test_size = 0.2)

In [8]:
# tr_shape = ((Xtr.shape[0],Xtr.shape[1]*Xtr.shape[2]))
# ts_shape = ((Xts.shape[0],Xts.shape[1]*Xts.shape[2]))

# scaler = StandardScaler()
# Xtr_scaled = scaler.fit_transform(Xtr.reshape(tr_shape))
# Xts_scaled = scaler.transform(Xts.reshape(ts_shape))

In [9]:
# Xtr_pathname = "/Volumes/MasterDrive/ML project data/Xtr_scaled.pkl"
# with open(Xtr_pathname, 'wb') as output_Xtr:
#     pickle.dump(Xtr_scaled, output_Xtr, pickle.HIGHEST_PROTOCOL)
    
# Xts_pathname = "/Volumes/MasterDrive/ML project data/Xts_scaled.pkl"
# with open(Xts_pathname, 'wb') as output_Xts:
#     pickle.dump(Xts_scaled, output_Xts, pickle.HIGHEST_PROTOCOL)

In [10]:
# pca = PCA()
# Xtr_reduced = pca.fit_transform(Xtr_scaled.reshape(tr_shape))
# Xts_reduced = pca.transform(Xts_scaled.reshape(ts_shape))

In [11]:
# Xtr_pathname = "/Volumes/MasterDrive/ML project data/Xtr_reduced.pkl"
# with open(Xtr_pathname, 'wb') as output_Xtr_red:
#     pickle.dump(Xtr_reduced, output_Xtr_red, pickle.HIGHEST_PROTOCOL)
    
# Xts_pathname = "/Volumes/MasterDrive/ML project data/Xts_reduced.pkl"
# with open(Xts_pathname, 'wb') as output_Xts_red:
#     pickle.dump(Xts_reduced, output_Xts_red, pickle.HIGHEST_PROTOCOL)

In [12]:
# ytr_pathname = "/Volumes/MasterDrive/ML project data/ytr.pkl"
# with open(ytr_pathname, 'wb') as output_ytr:
#     pickle.dump(ytr, output_ytr, pickle.HIGHEST_PROTOCOL)

# yts_pathname = "/Volumes/MasterDrive/ML project data/yts.pkl"
# with open(yts_pathname, 'wb') as output_yts:
#     pickle.dump(yts, output_yts, pickle.HIGHEST_PROTOCOL)

In [1]:
import os
import pickle

# Xtr and Xts are the processed data that have been modified 
# via scaling and PCA.

Xtr_pathname = str(os.getcwd()) + "/Xtr_reduced.pkl"
with open(Xtr_pathname, 'rb') as infile_Xtr:
    Xtr_reduced = pickle.load(infile_Xtr)

Xts_pathname = str(os.getcwd()) + "/Xts_reduced.pkl"
with open(Xts_pathname, 'rb') as infile_Xts:
    Xts_reduced = pickle.load(infile_Xts)
    
ytr_pathname = str(os.getcwd()) + "/ytr.pkl"
with open(ytr_pathname, 'rb') as infile_ytr:
    ytr = pickle.load(infile_ytr)
    
yts_pathname = str(os.getcwd()) + "/yts.pkl"
with open(yts_pathname, 'rb') as infile_yts:
    yts = pickle.load(infile_yts)

In [2]:
ytr.shape

(3834,)

In [3]:
from sklearn.utils import resample
import numpy as np

Xtr_majority = Xtr_reduced[ytr==0]
n_class = Xtr_majority.shape[0]
Xtr_minority = Xtr_reduced[ytr==1]

Xtr_minority_upsampled = resample(Xtr_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=n_class,    # to match majority class
                                 random_state=123) # reproducible results

ytr_majority = ytr[ytr==0]
ytr_minority_upsampled = np.ones(n_class)

Xtr_upsampled = np.concatenate([Xtr_majority,Xtr_minority_upsampled])
ytr_upsampled = np.concatenate([ytr_majority,ytr_minority_upsampled]).astype(int)

permutation = np.random.permutation(Xtr_upsampled.shape[0])

Xtr_upsampled = Xtr_upsampled[permutation]
ytr_upsampled = ytr_upsampled[permutation]

# no need to upsampled test data because we only need a balanced dataset to train.

In [4]:
# from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(Xtr_upsampled, ytr_upsampled)

yhat = model.predict(Xts_reduced)

/Users/alexhuang/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [5]:
from sklearn.metrics import f1_score, precision_score, recall_score

# F1 score is more appropriate due to imbalanced dataset
print(f1_score(yts,yhat))
print(precision_score(yts,yhat))
print(recall_score(yts,yhat))

0.3393213572854291
0.25449101796407186
0.5089820359281437
